# How to perform chemical equilibrium calculations with fixed volume and internal energy

Let's consider again the combustion of 1 mol of CH{{_4}} and 0.5 mol of O{{_2}}. This time, however, we will not specify temperature and pressure, or temperature and volume, but **volume and internal energy**. This is to model the combustion of this gas mixture in a rigid and adiabatic chamber. In this problem, both temperature and pressure are unknowns and will be calculated along with the amounts of the species that bring the system of a state of chemical equilibrium:

In [1]:
from reaktoro import *

db = NasaDatabase("nasa-cea")

gases = GaseousPhase("CH4 O2 CO2 CO H2O H2")

system = ChemicalSystem(db, gases)

state0 = ChemicalState(system)
state0.temperature(25.0, "celsius")
state0.pressure(1.0, "bar")
state0.set("CH4", 1.0, "mol")
state0.set("O2",  0.5, "mol")

props0 = ChemicalProps(state0)

V0 = props0.volume()  # the initial volume of the gases
U0 = props0.internalEnergy()  # the initial internal energy of the gases

In [2]:

specs = EquilibriumSpecs(system)
specs.volume()
specs.internalEnergy()

conditions = EquilibriumConditions(specs)
conditions.volume(V0)
conditions.internalEnergy(U0)

conditions.setLowerBoundPressure(1.0, "bar")
conditions.setLowerBoundTemperature(25.0, "celsius")

state = ChemicalState(state0)

solver = EquilibriumSolver(specs)
solver.solve(state, conditions)

print(state)

+-----------------+-----------+------+
| Property        |     Value | Unit |
+-----------------+-----------+------+
| Temperature     |   1230.15 |    K |
| Pressure        |    768530 |   Pa |
| Charge:         |         0 |  mol |
| Element Amount: |           |      |
| :: H            |         4 |  mol |
| :: C            |         1 |  mol |
| :: O            |         1 |  mol |
| Species Amount: |           |      |
| :: CH4          |  0.102994 |  mol |
| :: O2           |     1e-16 |  mol |
| :: CO2          | 0.0261655 |  mol |
| :: CO           |   0.87084 |  mol |
| :: H2O          | 0.0768286 |  mol |
| :: H2           |   1.71718 |  mol |
+-----------------+-----------+------+





Let's see different use cases for {{EquilibriumSolver}} in the next sections.

## Checking if the calculation was successful

Performing chemical equilibrium calculations is not a trivial task. It involves highly complicated algorithms to solve different mathematical problems (eg non-linear equations, matrix equations). Therefore, it is possible that a calculation could fail. This can happen due to many factors. However, the most common are:

* the formulation of the equilibrium problem is ill-formed (e.g. specifying a set of conflicting equilibrium constraints that cannot be achieved chemically/thermodynamically)
* the use of poor initial guesses.

So you might want to check at the end of each equilibrium calculation if it was successful:

In [3]:
state = ChemicalState(system)
state.temperature(1000, "celsius")
state.pressure(100, "bar")
state.set("CH4", 1.0, "mol")
state.set("O2",  0.5, "mol")

solver = EquilibriumSolver(system)
result = solver.solve(state)

print("Succeesful?", result.optima.succeeded)

Succeesful? True


The object `result` above is of type {{EquilibriumResult}} and it also contains other details about the calculation, such as the number of iterations needed for convergence:

In [4]:
print("Iterations:", result.optima.iterations)

Iterations: 27


## Calculations with given initial element amounts